In [2]:
# 1. Clone the converter repo into working dir
!git clone https://github.com/Taeyoung96/Yolo-to-COCO-format-converter.git 

# 2. Install dependencies
!pip install numpy opencv-python imagesize


Cloning into 'Yolo-to-COCO-format-converter'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 278 (delta 46), reused 47 (delta 35), pack-reused 208 (from 1)
Receiving objects: 100% (278/278), 30.92 MiB | 4.32 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [1]:
!conda create -n Yolo-to-COCO -y python=3.8
!pip install opencv-python imagesize


Retrieving notices: ...working... done
Solving environment: unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.3.1



## Package Plan ##

  environment location: /Users/manaspatil/anaconda3/envs/Yolo-to-COCO

  added / updated specs:
    - python=3.8


The following NEW packages will be INSTALLED:

  ca-certificates    pkgs/main/osx-arm64::ca-certificates-2025.2.25-hca03da5_0 
  libcxx             pkgs/main/osx-arm64::libcxx-14.0.6-h848a8c0_0 
  libffi             pkgs/main/osx-arm64::libffi-3.4.4-hca03da5_1 
  ncurses            pkgs/main/osx-arm64::ncurses-6.4-h313beb8_0 
  openssl            pkgs/main/osx-arm64::openssl-3.0.16-h02f6b

In [4]:
%cd /Users/manaspatil/Desktop/Acads/DAC-204/Label-converter/Yolo-to-COCO-format-converter
!python main.py --path /Users/manaspatil/Desktop/Acads/DAC-204/Label-converter/images --output output.json


/Users/manaspatil/Desktop/Acads/DAC-204/Label-converter/Yolo-to-COCO-format-converter
Start!
Processing 31679 ...Finished!


In [14]:
import os
from pathlib import Path

def validate_and_clean_dataset(image_dir, label_dir, valid_classes):
    """Clean dataset by removing invalid YOLO labels and their corresponding images"""
    image_dir = Path(image_dir)
    label_dir = Path(label_dir)
    
    def is_valid_label(label_path):
        """Check if label file meets YOLO format requirements"""
        try:
            with open(label_path, 'r') as f:
                for line_num, line in enumerate(f, 1):
                    parts = line.strip().split()
                    # Check 5 elements per line
                    if len(parts) != 5:
                        print(f"Invalid line {line_num} in {label_path.name}: {line.strip()}")
                        return False
                    # Validate class ID
                    try:
                        class_id = int(parts[0])
                        if class_id not in valid_classes:
                            print(f"Invalid class ID {class_id} in {label_path.name}")
                            return False
                    except ValueError:
                        print(f"Non-integer class ID in {label_path.name}: {parts[0]}")
                        return False
                    # Validate coordinates are floats in [0,1]
                    try:
                        coords = list(map(float, parts[1:]))
                        if any(not (0.0 <= x <= 1.0) for x in coords):
                            print(f"Invalid coordinates in {label_path.name}: {coords}")
                            return False
                    except ValueError:
                        print(f"Non-float coordinates in {label_path.name}: {parts[1:]}")
                        return False
            return True
        except Exception as e:
            print(f"Error reading {label_path.name}: {str(e)}")
            return False

    # Find all invalid label files
    invalid_labels = []
    for label_file in label_dir.glob('*.txt'):
        if not is_valid_label(label_file):
            invalid_labels.append(label_file)
            print(f"Marked for deletion: {label_file.name}")

    # Delete invalid files and corresponding images
    deleted_files = {'labels': [], 'images': []}
    for label_file in invalid_labels:
        # Delete label file
        try:
            os.remove(label_file)
            deleted_files['labels'].append(label_file.name)
        except Exception as e:
            print(f"Error deleting {label_file.name}: {str(e)}")
        
        # Delete corresponding images
        image_stem = label_file.stem
        for ext in ['jpg', 'jpeg', 'png', 'bmp', 'webp']:  # Common image extensions
            image_file = image_dir / f"{image_stem}.{ext}"
            if image_file.exists():
                try:
                    os.remove(image_file)
                    deleted_files['images'].append(image_file.name)
                except Exception as e:
                    print(f"Error deleting {image_file.name}: {str(e)}")

    return deleted_files

# Configuration (update with your paths)
config = {
    "image_dir": r"/Users/manaspatil/Desktop/Acads/DAC-204/Label-converter/images",
    "label_dir": r"/Users/manaspatil/Desktop/Acads/DAC-204/Label-converter/labels",
    "valid_classes": set(range(12))  # 0-11 based on your category_map
}

# Run the cleaner
result = validate_and_clean_dataset(**config)

# Print summary
print("\nCleanup Summary:")
print(f"Deleted {len(result['labels'])} invalid label files")
print(f"Deleted {len(result['images'])} corresponding image files")
print("Invalid labels removed:", *result['labels'], sep='\n- ')
print("Images removed:", *result['images'], sep='\n- ')

Invalid coordinates in 07207.txt: [1.6140625, 1.7296875, 1.140625, 1.5078125]
Marked for deletion: 07207.txt
Invalid coordinates in 07213.txt: [1.49140625, 1.71171875, 1.11953125, 1.57578125]
Marked for deletion: 07213.txt
Invalid class ID 12 in 20848.txt
Marked for deletion: 20848.txt
Invalid coordinates in 07165.txt: [1.35625, 1.56484375, 1.21171875, 1.15390625]
Marked for deletion: 07165.txt
Invalid coordinates in 07159.txt: [1.3515625, 1.56328125, 1.221875, 1.16328125]
Marked for deletion: 07159.txt
Invalid class ID 12 in 20914.txt
Marked for deletion: 20914.txt
Invalid coordinates in 07373.txt: [1.36640625, 1.5546875, 1.23203125, 1.1171875]
Marked for deletion: 07373.txt
Invalid coordinates in 07367.txt: [1.35, 1.57265625, 1.2171875, 1.153125]
Marked for deletion: 07367.txt
Invalid coordinates in 07401.txt: [1.35390625, 1.5578125, 1.2171875, 1.140625]
Marked for deletion: 07401.txt
Invalid coordinates in 07429.txt: [1.3546875, 1.56640625, 1.21875, 1.17421875]
Marked for deletion: 